In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [2]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [4]:
print(examples_train[14000])

{'tokens': ['British', 'Airways', 'dropped', 'out', 'of', 'the', 'current', 'bidding', 'for', 'United', 'Air', "'s", 'parent', ',', 'leaving', 'a', 'UAL', 'management-pilots', 'group', 'without', 'a', 'key', 'partner', '.'], 'senid': 6, 'mentions': [{'start': 2, 'labels': ['/EVENT', '/EVENT/WAR'], 'end': 3}, {'start': 0, 'labels': ['/ORGANIZATION/CORPORATION', '/ORGANIZATION'], 'end': 2}, {'start': 0, 'labels': ['/GPE/COUNTRY', '/GPE', '/LOCATION'], 'end': 1}], 'fileid': 'WSJ1728'}


In [5]:
examples_iter = iter(examples_train[0:10])

In [6]:
import operator
import itertools
from tqdm import tqdm
# dictionary

restricted_set = set()
for i in tqdm(examples_iter):
  istance = i['mentions'][:]

  if len(istance)>1:
    d = sorted(istance, key=operator.itemgetter("start"))

    d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}
    all_list=[]
    for key, value in d.items():
      all_list.append(value)

    permutations = list(itertools.product(*all_list))
    print(permutations)

    iter = 0
    for p in permutations:
          
          mini_list = []
          for lista_labels in p:
            mini_list.append(lista_labels['labels'])

          permutations_2 = list(itertools.product(*mini_list))
          print(permutations_2)

          for p_2 in permutations_2:

            phrase = i['tokens'][:]
            print(phrase)
            print(p_2)
            
            for h in range(0, len(p_2)):
              print(permutations[iter][h]['end']-permutations[iter][h]['start'])
              if (permutations[iter][h]['end']-permutations[iter][h]['start']==1):          
                phrase[permutations[iter][h]['start']] = p_2[h]
                restricted_set.add(p_2[h])
              else:
                phrase.pop(permutations[iter][h]['end']-1)
                phrase[permutations[iter][h]['start']] = p_2[h]
                restricted_set.add(p_2[h])
            
            print(phrase)
            
          iter+=1
  else:
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(p_2[h])
      else:
        phrase.pop(i['mentions'][0]['end']-1)
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(p_2[h])

        print(phrase)
      



10it [00:00, 425.91it/s]

[({'start': 12, 'labels': ['/ORGANIZATION/CORPORATION', '/WORK_OF_ART', '/ORGANIZATION', '/WORK_OF_ART/BOOK'], 'end': 13}, {'start': 15, 'labels': ['/PLANT', '/PRODUCT', '/SUBSTANCE', '/SUBSTANCE/FOOD'], 'end': 16})]
[('/ORGANIZATION/CORPORATION', '/PLANT'), ('/ORGANIZATION/CORPORATION', '/PRODUCT'), ('/ORGANIZATION/CORPORATION', '/SUBSTANCE'), ('/ORGANIZATION/CORPORATION', '/SUBSTANCE/FOOD'), ('/WORK_OF_ART', '/PLANT'), ('/WORK_OF_ART', '/PRODUCT'), ('/WORK_OF_ART', '/SUBSTANCE'), ('/WORK_OF_ART', '/SUBSTANCE/FOOD'), ('/ORGANIZATION', '/PLANT'), ('/ORGANIZATION', '/PRODUCT'), ('/ORGANIZATION', '/SUBSTANCE'), ('/ORGANIZATION', '/SUBSTANCE/FOOD'), ('/WORK_OF_ART/BOOK', '/PLANT'), ('/WORK_OF_ART/BOOK', '/PRODUCT'), ('/WORK_OF_ART/BOOK', '/SUBSTANCE'), ('/WORK_OF_ART/BOOK', '/SUBSTANCE/FOOD')]
['The', 'harvest', 'arrives', 'in', 'plenty', 'after', 'last', 'year', "'s", 'drought-ravaged', 'effort', ':', 'The', 'government', 'estimates', 'corn', 'output', 'at', '7.45', 'billion', 'bushels',

In [ ]:
#open(path_corpus_data + 'corpus_data_bbn.txt', "w").write(' '.join(phrase) + '\n')

KeyboardInterrupt: ignored

In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_bbn.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)